# Seismic Data Exercise

## Exercise A: Download miniSEED files from dataselect

This exercise is a script for downloading seismic data from EarthScope in the miniSEED file format. All the functions are covered in the previous `4_seismic.ipynb` notebook. The download function accommodates either a SAGE web service request or GAGE web service request. Fill in the blanks to complete the script.

Download earthquake event data by the query parameters for the SAGE web service. You can use the example in the previous notebook or use another event and station. 

If the script is correctly completed you will have a new directory called `/data` containing a miniSEED file.

Fill in the blanks as required.

In [ ]:
# ---------- Fill in the package imports for this script ----------
import ____  
from datetime import datetime
from pathlib import Path
import ____

# ---------- Create an EarthScope client to get token ----------
client = ____


# Fill this with the service endpoint for SAGE data
SAGE_URL = ____

DATA_DIR = Path("./data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# ---------- Get an EarthScope token ----------
def get_token():
    """
    Use the SDK to refresh (if needed) and return an access token.
    """
    # Refresh if necessary
    ____

    # Retrieve the token string
    token = ____
    return token


# ---------- Helper: common auth header ----------
def auth_header(token):
    """
    Helper function to create header with token
    """
    # Use a standard Bearer token header (lowercase 'authorization' is fine)
    return ____


# ---------- Download a file from EarthScope's web services ----------
# --------------------------------------------------------------------
def download_data(url, data_directory, params={}):  # params is an optional query parameter
    """
    Sends GET with query parameters to the SAGE web service and saves the response 
    body to a file. Expected params include: net, sta, loc, cha, start (ISO), end (ISO), etc.
    """
    # get authorization token
    token = get_token()

    # create the authorization header 
    header = auth_header(token) 


    # This creates a file name for a miniSEED file by
    # parsing the start datetime to build filename parts
    start_dt = datetime.strptime(params["start"], "%Y-%m-%dT%H:%M:%S")
    year = str(start_dt.year) # Day-of-year (001-366) for file naming
    doy =  "{:03d}".format(____)  # hint: see how year is extracted from the start
    
    # create a file name for a miniSEED
    file_name = ".".join([
        params["sta"],
        params["net"],
        params["loc"],
        params["cha"],
        year,
        doy,
        "mseed"
    ])
    
    # create a variable with the path to the data directory and file
    out_path = Path(Path(data_directory) / ____)

    # Make the request to the web service with params and bearer auth
    r = requests.____(
        url,
        params=____,
        headers=____,
        stream=True
    ) 

    if r.status_code == requests.codes.____:
        with open(out_path, "wb") as f:
            for data in r:
                f.write(data)
    else:
        #problem occured
        print(f"failure: {r.status_code}, {r.reason}")
        return None

# Download a miniSEED file for an event
# 
params = {"net" : 'IU',
          "sta" : 'ANMO',
          "loc" : '00',
          "cha" : 'BHZ',
          "start": '2010-02-27T06:30:00',
          "end": '2010-02-27T10:30:00'}

# Try the parameterized request
download_data(____, DATA_DIR, ___)

# Download a RINEX file
year = 2025
day = 1
station = 'p034'
doy = '%03d'.format(day)
compression = 'd.Z'  # or ".Z" / "" depending on the archive
url = create_url(year, ____, station, compression)
download_data(url, ____)

## Exercise B: Access Seismic Data from an AWS S3 Bucket

This exercise shows how to get data from an AWS S3 bucket, stream the data to obspy, and print out the metadata for a trace. Fill in the blanks as needed.

In [ ]:
# --- Fill in the blanks (marked ____ ) ---
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import io
from obspy import read

# 1) Create an anonymous S3 client in the us-west-2 region
s3 = boto3.client('s3',
                  config=Config(signature_version=____),
                  region_name='____')

# 2) Identify what to fetch
BUCKET_NAME = '____'   # public bucket name
KEY = '____'           # object key/path inside the bucket

# 3) Download the object and put bytes into a buffer
response = s3.get_object(Bucket=____, Key=____)
data_stream = io.BytesIO(response['Body'].read())

# 4) Parse the miniSEED bytes with ObsPy
st = read(____)

# 5) Print a summary of the Stream
print(st)

# 6) (bonus) Print the first Trace's stats: network, station, channel, starttime, sampling_rate
tr = st[0]
print(tr.stats.network, tr.stats.station, tr.stats.channel, tr.stats.starttime, tr.stats.sampling_rate)


## Answer Key for Exercise A

```{admonition} Click to see answer
:class: dropdown

<PRE>
# ---------- Imports ----------
import requests
import os
from datetime import datetime
from pathlib import Path
from earthscope_sdk import EarthScopeClient

# ---------- Create an EarthScope client to get token ----------
client = EarthScopeClient()

# Fill this with the service endpoint for SAGE data
SAGE_URL = "http://service.iris.edu/fdsnws/dataselect/1/query?"

# create a directory for rinex data
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

# ---------- Get an EarthScope token ----------
def get_token():
    """
    Use the SDK to refresh (if needed) and return an access token.
    """
    # Refresh if necessary
    client.ctx.auth_flow.refresh_if_necessary()

    # Retrieve the token string
    token = client.ctx.auth_flow.access_token
    return token


# ---------- Helper: common auth header ----------
def auth_header(token):
    # Use a standard Bearer token header (lowercase 'authorization' is fine)
    return {"authorization": f"Bearer {token}"}


# ---------- Download a file from EarthScope's web services ----------
# --------------------------------------------------------------------
def download_data(url, data_directory, params={}):
    """
    Sends GET with query parameters and saves the response body to a file.
    Expected params include: net, sta, loc, cha, start (ISO), end (ISO), etc.
    """
    # get authorization token
    token = get_token()

    # create the authorization header 
    header = auth_header(token) 
    
    # Example filename: STA.NET.LOC.CHA.YEAR.DOY.mseed
    start_dt = datetime.strptime(params["start"], "%Y-%m-%dT%H:%M:%S")
    year = str(start_dt.year) # Day-of-year (001-366) for file naming
    doy =  "{:03d}".format(start_dt.day)  # hint: see how year is extracted from the start
    file_name = ".".join([
        params["sta"],
        params["net"],
        params["loc"],
        params["cha"],
        year,
        doy,
        "mseed"
    ])

    out_path = Path(Path(data_directory) / file_name)

    # Make the request with params and bearer auth
    r = requests.get(
        url,
        params=params,
        headers=header,
        stream=True
    ) 

    if r.status_code == requests.codes.ok:
        with open(out_path, "wb") as f:
            for data in r:
                f.write(data)
    else:
        #problem occured
        print(f"failure: {r.status_code}, {r.reason}")
        return None

# Example query parameters (adjust to a valid service)
params = {"net" : 'IU',
          "sta" : 'ANMO',
          "loc" : '00',
          "cha" : 'BHZ',
          "start": '2010-02-27T06:30:00',
          "end": '2010-02-27T10:30:00'}

# Try the parameterized request
download_data(SAGE_URL, DATA_DIR, params)
</PRE>
```

## Answer Key for Exercise B

```{admonition} Click to see answer
:class: dropdown

<PRE>
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import io
from obspy import read

s3 = boto3.client('s3',
                  config=Config(signature_version=UNSIGNED),
                  region_name='us-west-2')

BUCKET_NAME = 'ncedc-pds'
KEY = 'continuous_waveforms/BK/2014/2014.236/PACP.BK.HHN.00.D.2014.236'

response = s3.get_object(Bucket=BUCKET_NAME, Key=KEY)
data_stream = io.BytesIO(response['Body'].read())

st = read(data_stream)
print(st)

tr = st[0]
print(tr.stats.network, tr.stats.station, tr.stats.channel, tr.stats.starttime, tr.stats.sampling_rate)
</PRE>
```

## [< Previous](./4_seismic_data.ipynb)&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;[Next >](./6_geodetic_data.ipynb)